In [17]:
## Load the model
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.messages import HumanMessage

query_text = 'how should I take care of my parents with Alzheimer disease?' # failed, no results
user_id = 'John Snow'

GENERATION_PROMPT = """
Based on the provided context, answer the user's question. If the context doesn't contain enough information or your not sure, say so.

Context:
{context}

Question: {question}

Return a clear and concise answer. The output should be in the following format:

Answer: <answer>
Sources: <sources>

If context isn't provided, use "Answer" should start with "Sorry, I don't have enough information to answer that question." and "Sources" should be empty.
"""

memories = mem0.search(query_text, user_id)

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=GENERATION_PROMPT
)

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    HumanMessage(content=query_text),
    # ("system", prompt)
    
])

# query_text = 'what is the specialty for Chinese language. ' 

local_llm = 'marco-o1'
local_embedding_model = 'BAAI/bge-m3'

llm = ChatOllama(model=local_llm, temperature=0.0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0.0, format='json')

### Initialize Vector store and embedding model

Get peer support knowledge base and research knowledge base

---------------------------

In [ ]:
from embedding.vector_store import VectorStore
from embedding.embedding_models import EmbeddingModels

embedding_model = EmbeddingModels().get_bge_embedding(model_name=local_embedding_model)
vectorstore = VectorStore()

peer_kb = vectorstore.get_chroma_vectorstore(
    vectorstore_path='./data/vector_database/peer_kb', 
    embedding=embedding_model
)

research_kb = vectorstore.get_chroma_vectorstore(
    vectorstore_path='./data/vector_database/research_kb', 
    embedding=embedding_model
)

In [11]:
from checkpoints.routering import get_routing_decision

intention_detection = get_routing_decision(query_text, model=local_llm)

AttributeError: 'str' object has no attribute 'content'

In [ ]:


similarity_search_res = vectorstore.retrieve_docs(query=query_text, vectorstore=, k=5)

for i, doc in enumerate(similarity_search_res):
    print(f"---------------------------||Document Number: {i} ||------------------------ ")
    print(doc.page_content)
    print(doc.metadata)
    print("\n")

### Retrieve grading 
*Preventing hallucination and erroneous retrieval that is not relevant to the question but based on the idiosyncrasies of the embedding model or chunking.*

---------------------

In [ ]:
from typing import List
from checkpoints.retrieval_grading import grade_retrieval
from langchain.schema import Document

# Grade the retrieval results
graded_docs = grade_retrieval(query_text, similarity_search_res, model="llama3.2")

# Print graded results
# for doc in graded_docs:
#     print(f"Document content: {doc}")
    
# Filter out documents with relevance_score lower than 0.5
class ReasonedDocument:
    """
    A class to store the document, relevance score, and reasoning.
    """
    
    document: Document # Retrieved document from vector store in Document type
    relevance_score: float # Relevance score by reasoning model
    reasoning: str # Reasons for the relevance score
    
    def __init__(self, document, relevance_score, reasoning):
        self.document = document
        self.relevance_score = relevance_score
        self.reasoning = reasoning
        
    def __str__(self):
        return f"Document: {self.document}\nRelevance Score: {self.relevance_score}\nReasoning: {self.reasoning}\n"

# filtered_docs = [doc for doc, grade in zip(similarity_search_res, graded_docs) if grade["relevance_score"] >= 0.5]

filtered_docs = []

# Filter out documents with relevance_score lower than 0.5 and re-rank relevant documents based on relevance_score descending
filtered_docs = []

for doc, grade in zip(similarity_search_res, graded_docs):
    if grade["relevance_score"] >= 0.5:
        reasoned_doc = ReasonedDocument(
            document=doc,
            relevance_score=grade["relevance_score"],
            reasoning=grade["reasoning"]
        )
        filtered_docs.append(reasoned_doc)

# Re-rank the filtered documents based on relevance_score in descending order
filtered_docs.sort(key=lambda x: x.relevance_score, reverse=True)

# Print filtered results
for doc in filtered_docs:
    print(doc)
    # print(doc.metadata)



### Generate answer

*Using the graded documents to generate an answer.*

1. Only use the graded documents to generate an answer.

--------------------------------

In [ ]:
from answer_generation import generate_answer

context_chunks = [doc.document for doc in filtered_docs]

# TODO: handle the case where context_chunks is empty, answer should specify that we don't have relevant information.

answer = generate_answer(query_text, context_chunks, model="marco-o1")

print(answer)